In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

# =====================
# Ler CSV
# =====================
df = pd.read_csv("area_feicoes_todos_anos.csv", delimiter=';')
df["geometry"] = df["geometry"].apply(wkt.loads)

gdf = gpd.GeoDataFrame(df, geometry="geometry")
gdf = gdf.set_crs(epsg=31982)

In [ ]:
import geopandas as gpd

gdf['nome'] = gdf['nome'].replace({
    'SACO DOS LIM�ES': 'SACO DOS LIMÕES',
    'LAGOA DA CONCEI��O': 'LAGOA DA CONCEIÇÃO',
    'RIBEIR�O DA ILHA': 'RIBEIRÃO DA ILHA',
    'P�NTANO DO SUL': 'PÂNTANO DO SUL',
    'SANTO ANT�NIO DE LISBOA': 'SANTO ANTÔNIO DE LISBOA'
})

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import contextily as cx
from matplotlib.gridspec import GridSpec
from matplotlib_scalebar.scalebar import ScaleBar
from pathlib import Path

distritos = gdf['nome'].unique()
for distrito in distritos:
    # =====================
    # Filtro
    # =====================
    # =====================
    # LEGENDA PADRÃO DO DISTRITO
    # =====================
    gdf_distrito = gdf[gdf["nome"] == distrito].copy()
    gdf_distrito = gdf_distrito.to_crs(epsg=3857)
    gdf_distrito['area_m2'] = gdf_distrito.geometry.area
    
    legenda_padrao = (
        gdf_distrito
        .groupby(['classe', 'subclasse', 'hex'], dropna=False)['area_m2']
        .sum()
        .reset_index()
    )
    
    legenda_padrao['classe'] = legenda_padrao['classe'].fillna('Sem classe')
    legenda_padrao['subclasse'] = legenda_padrao['subclasse'].fillna('Sem subclasse')
    legenda_padrao['cor_plot'] = legenda_padrao['hex'].fillna('#000000')
    
    # Ordem fixa (por classe → subclasse)
    legenda_padrao = legenda_padrao.sort_values(['classe', 'subclasse'])
    
    # Labels FIXOS (sem área)
    legenda_padrao['label_legenda'] = (
        legenda_padrao['classe'] + ' - ' +
        legenda_padrao['subclasse']
    )
    
    for ano in range(1985, 2025):  
        gdf_filtrado = (
            gdf[(gdf["nome"] == distrito) & (gdf["ano"] == ano)]
            .copy()
        )
        
        # =====================
        # Preparar dados visuais
        # =====================
        gdf_filtrado['cor_plot'] = gdf_filtrado['hex'].fillna('#000000')
        
        gdf_filtrado['legenda'] = (
            gdf_filtrado['classe'].fillna('Sem classe').astype(str)
            + ' - ' +
            gdf_filtrado['subclasse'].fillna('Sem subclasse').astype(str)
        )
        
        # =====================
        # Reprojetar para basemap
        # =====================
        gdf_web = gdf_filtrado.to_crs(epsg=3857)
        
        # =====================
        # Tabela-resumo
        # =====================
        gdf_web['area_m2'] = gdf_web.geometry.area
        
        # =====================
        # Área por subclasse (para legenda)
        # =====================
        legenda_area = (
            gdf_web
            .groupby(['classe', 'subclasse', 'cor_plot'], dropna=False)['area_m2']
            .sum()
            .reset_index()
        )
        
        legenda_area['classe'] = legenda_area['classe'].fillna('Sem classe')
        legenda_area['subclasse'] = legenda_area['subclasse'].fillna('Sem subclasse')
        legenda_area['area_km2'] = legenda_area['area_m2'] / 1e6
        
        # Texto final da legenda
        legenda_area['label_legenda'] = (
            legenda_area['classe'] + ' - ' +
            legenda_area['subclasse'] +
            ' (' + legenda_area['area_km2'].map('{:.2f}'.format) + ' km²)'
        )
        
        tabela = (
            gdf_web
            .groupby('classe', dropna=False)['area_m2']
            .sum()
            .reset_index()
        )
        
        tabela['classe'] = tabela['classe'].fillna('Sem classe')
        tabela['area_km2'] = tabela['area_m2'] / 1e6
        tabela['percentual'] = 100 * tabela['area_km2'] / tabela['area_km2'].sum()
        
        tabela_display = tabela[['classe', 'area_km2', 'percentual']].copy()
        tabela_display['area_km2'] = tabela_display['area_km2'].map('{:.2f}'.format)
        tabela_display['percentual'] = tabela_display['percentual'].map('{:.1f}%'.format)
    
        
        # =====================
        # Layout
        # =====================
        fig = plt.figure(figsize=(13, 8))
        gs = GridSpec(1, 2, width_ratios=[2.2, 1], figure=fig)
        
        ax_map = fig.add_subplot(gs[0])
        ax_side = fig.add_subplot(gs[1])
        ax_side.axis('off')
        
        # =====================
        # Mapa
        # =====================
        gdf_web.plot(
            ax=ax_map,
            color=gdf_web['cor_plot'],
            edgecolor='black',
            linewidth=0.6,
            alpha=0.85
        )
        
        cx.add_basemap(
            ax_map,
            source=cx.providers.CartoDB.Positron,
            zoom=14
        )
        
        scalebar = ScaleBar(
            dx=1,
            units='m',
            location='lower right',
            length_fraction=0.25,
            box_alpha=0.6
        )
        
        ax_map.add_artist(scalebar)
        
        ax_map.set_title(
            f"Uso do Solo – {distrito} {ano}",
            fontsize=14,
            fontweight='bold',
            pad=12
        )
        ax_map.set_axis_off()
        
    
        # =====================
        # PAINEL DE LEGENDA FIXO
        # =====================
        # =====================
        # Legenda (EIXO FIXO)
        # =====================
        ax_legend = ax_side.inset_axes([0.02, 0.52, 0.96, 0.45])
        ax_legend.axis('off')
        
        legend_items = [
            mpatches.Patch(color=row.cor_plot, label=row.label_legenda)
            for row in legenda_padrao.itertuples(index=False)
        ]
        
        leg = ax_side.legend(
            handles=legend_items,
            title='Legenda: Classe - Subclasse',
            loc='upper left',
            bbox_to_anchor=(0.0, 0.98),   # 👈 desce a legenda
            bbox_transform=ax_side.transAxes,
            fontsize=9,
            title_fontsize=11,
            frameon=True
        )
        
        leg.get_title().set_fontweight('bold')
    
        # =====================
        # Tabela
        # =====================
        
        ax_table = ax_side.inset_axes([0.02, 0.05, 0.96, 0.45])
        ax_table.axis('off')
        
        if tabela_display.empty:
            ax_table.text(
                0.5, 0.5,
                "Sem dados para tabela",
                ha='center',
                va='center',
                fontsize=9,
                style='italic'
            )
        else:
            table = ax_table.table(
                cellText=tabela_display.values,
                colLabels=['Classe', 'Área (km²)', '%'],
                cellLoc='left',
                loc='center',
                colWidths=[0.55, 0.25, 0.20]
            )
        
        # Cabeçalho em negrito + alinhamento
        for (row, col), cell in table.get_celld().items():
            if row == 0:
                cell.set_text_props(fontweight='bold')
            else:
                if col in [1, 2]:
                    cell._loc = 'right'
    
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.scale(1, 1.4)
    
        # Cabeçalho em negrito
        for (row, col), cell in table.get_celld().items():
            if row == 0:
                cell.set_text_props(fontweight='bold')
            else:
                # Alinhar números à direita
                if col in [1, 2]:
                    cell._loc = 'right'
        
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.scale(1, 1.4)
        
        
        ax_table.set_title(
            'Resumo por classe (área)',
            fontsize=11,
            fontweight='bold',
            pad=0.5    # 👈 aproxima o título da tabela
        )
        
        fig.text(
            0.5, 0.05,   # 👈 mais baixo que antes
            "Autor: Theo G. Miqueluzzi | Dados: MapBiomas | Data: Janeiro 2026",
            ha='center',
            fontsize=9,
            style='italic'
        )
        
        plt.tight_layout(rect=[0, 0.04, 1, 1])
        
        # Caminho da pasta
        output_dir = Path(f"Imagens/{distrito}")
        
        # Cria a pasta (e pais) se não existir
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # Salvar imagem
        plt.savefig(
            output_dir / f"Uso do Solo - {distrito} {ano}.png",
            dpi=300,
            bbox_inches='tight'
        )
        
        plt.show()